### Importing Libraries and Dataset

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import string
import requests

In [2]:
data_link = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')

### Data Preprocessing

In [3]:
data = data_link.text.split('\n')

In [4]:
data[253]

'  From fairest creatures we desire increase,'

In [5]:
# We cut off the preface, foreword, copyright instructions from the file.
df = data[253:]

In [6]:
len(df)

124204

In [7]:
# Here we make the data set 'df' continous
df = ' '.join(df)

* The below function will remove the punctuations, keep only the alpha-numeric characters and convert all alphabets into lower case.

In [8]:
def text(doc):
    tokens = doc.split()
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens


In [9]:
tokens = text(df)

In [10]:
len(tokens)

898199

In [11]:
len(set(tokens))

27956

In [12]:
length = 50 + 1
lines = []

In [13]:
for i in range(length, len(tokens)):
    seq = tokens[i - length:i]
    line = ' '.join(seq)
    lines.append(line)
    if i > 250000:
        break

In [14]:
lines[0]

'from fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self'

In [15]:
lines[2]

'creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self thy foe'

## Working of model:
* Firstly we have successfully converted the given data into lines with 51 words each.
* What will do next is that, we split the data set 'lines' into x, y. x contains the first 50 words of each element of 'lines' data set, the y variable contains the 51st word of each element in 'lines'.
* Note that x will be a 2D array, while y will be a 1D array.
* Before split data into x, y we will tokenize the data, i.e. each distinct word will be assigned a distinct real number.
* We then split the x, y variables into train and test sets and then build the LSTM model.

In [16]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
word_seq = tokenizer.texts_to_sequences(lines)

In [17]:
word_seq = np.array(word_seq)

In [18]:
x, y = word_seq[:, : -1], word_seq[:, -1]

In [19]:
len(x)

249951

In [20]:
x_train = x[:200000, :]
y_train = y[:200000]
x_test = x[200000:, :]
y_test = y[200000:]

In [21]:
len(x_train), len(x_test)

(200000, 49951)

In [22]:
vocab = len(tokenizer.word_index) + 1

In [23]:
y_train = to_categorical(y_train, num_classes = vocab)
y_test = to_categorical(y_test, num_classes = vocab)

In [24]:
seq_len = x.shape[0]

## LSTM Model

In [27]:
model_rnn = Sequential()
model_rnn.add(Embedding(vocab, 50, input_length = seq_len))
model_rnn.add(LSTM(256, return_sequences = True))
model_rnn.add(LSTM(256))
model_rnn.add(Dense(256, activation = 'relu'))
model_rnn.add(Dense(vocab, activation = 'softmax'))

In [28]:
model_rnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 249951, 50)        743700    
_________________________________________________________________
lstm (LSTM)                  (None, 249951, 256)       314368    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dense_1 (Dense)              (None, 14874)             3822618   
Total params: 5,471,790
Trainable params: 5,471,790
Non-trainable params: 0
_________________________________________________________________


In [29]:
model_rnn.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [32]:
rnn_fit = model_rnn.fit(x_train, y_train, batch_size = 2000, epochs = 100)

Epoch 1/100
100/100 [==============================] - 650s 6s/step - loss: 6.7529 - accuracy: 0.0295
Epoch 2/100
100/100 [==============================] - 638s 6s/step - loss: 6.6376 - accuracy: 0.0335
Epoch 3/100
100/100 [==============================] - 643s 6s/step - loss: 6.5023 - accuracy: 0.0419
Epoch 4/100
100/100 [==============================] - 645s 6s/step - loss: 6.4125 - accuracy: 0.0442
Epoch 5/100
100/100 [==============================] - 645s 6s/step - loss: 6.3296 - accuracy: 0.0484
Epoch 6/100
100/100 [==============================] - 645s 6s/step - loss: 6.2556 - accuracy: 0.0542
Epoch 7/100
100/100 [==============================] - 642s 6s/step - loss: 6.1954 - accuracy: 0.0585
Epoch 8/100
100/100 [==============================] - 646s 6s/step - loss: 6.1397 - accuracy: 0.0621
Epoch 9/100
100/100 [==============================] - 649s 6s/step - loss: 6.0847 - accuracy: 0.0661
Epoch 10/100
100/100 [==============================] - 644s 6s/step - loss: 6.049

100/100 [==============================] - 646s 6s/step - loss: 3.0171 - accuracy: 0.3573
Epoch 78/100
100/100 [==============================] - 643s 6s/step - loss: 2.9948 - accuracy: 0.3614
Epoch 79/100
100/100 [==============================] - 652s 7s/step - loss: 2.9726 - accuracy: 0.3658
Epoch 80/100
100/100 [==============================] - 648s 6s/step - loss: 2.9509 - accuracy: 0.3680
Epoch 81/100
100/100 [==============================] - 644s 6s/step - loss: 2.9317 - accuracy: 0.3724
Epoch 82/100
100/100 [==============================] - 652s 7s/step - loss: 2.9073 - accuracy: 0.3757
Epoch 83/100
100/100 [==============================] - 646s 6s/step - loss: 2.8893 - accuracy: 0.3787
Epoch 84/100
100/100 [==============================] - 645s 6s/step - loss: 2.8686 - accuracy: 0.3817
Epoch 85/100
100/100 [==============================] - 648s 6s/step - loss: 2.8469 - accuracy: 0.3855
Epoch 86/100
100/100 [==============================] - 648s 6s/step - loss: 2.8266 - 

In [33]:
model_rnn.save('text_rnn.h5')